# Lecture 16 (5/2/2022)

**Announcements**



*Last time we covered:*
- Linear regression
    - Simple linear regression
    - Evaluating regression
    - Polynomial and multiple regression

**Today's agenda:**
- Multiple regression continued
- Overfitting


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Multiple Regression


## Big idea: multiple predictors for $y$

In real-world scenarios, it's common to explore the relationship between *multiple* predictor variables that all jointly impact the dependent variable. 

For this, we need **multiple regression**. 

Though the guiding principles are similar to *simple regression*, things can get much more complicated in multiple dimensions. Here, we'll provide enough of an overview for you to be able to use multiple regression as part of your modeling toolbox.

## Example: predicting life expectancy

Let's start with a familiar example.

In Problem Set 3 and elsewhere, we've looked at the *gapminder* dataset, which explores the relationship between GDP, population, and life expectancy at the national level and over time. 

In [ ]:
gap = pd.read_csv("https://raw.githubusercontent.com/UCSD-CSS-002/ucsd-css-002.github.io/master/datasets/gapminder.csv")

# Let's keep just some of the variables (note for pset!)
gap_subset = gap.loc[gap['year'] == 2007, ('country', 'year', 'lifeExp', 'pop', 'gdpPercap')]

# Add log population
gap_subset['logPop'] = np.log10(gap_subset['pop'])
gap_subset['logGdpPercap'] = np.log10(gap_subset['gdpPercap'])
gap_subset

In Problem Set 3, you generated a graph that explored the relationship between income (GDP per-capita on $x$) and life expectancy (in years on $y$), alongside two additional predictors: region (color) and population (size).

![gap](img/gapminder.png)

Here, we want to know formally what the relationship is between the three continuous variables (income, life expectancy, and population). In other words, **can we predict life expectancy using both income *and* population better than we could only using one of those variables?**

Let's start by just examining each of the predictors in isolation to see if this is a plausible hypothesis.

In [ ]:
g = sns.lmplot(data = gap_subset, 
                    x = "logGdpPercap", # x1 
                    y = "lifeExp"
                   )
plt.title("Life expectancy as a function of income")
plt.xlabel("Income (log GDP / capita)")
plt.ylabel("Life expectancy")
plt.show()

h = sns.lmplot(data = gap_subset, 
                    x = "logPop", # x2 
                    y = "lifeExp"
                   )
plt.title("Life expectancy as a function of population")
plt.xlabel("Population (log)")
plt.ylabel("Life expectancy")
plt.show()

One of these variables has a strong positive relationship. The other one seems a bit less clear. 

How can we think about exploring the role that both of them play together in predicting life expectancy?

### Multiple regression: overview

Multiple regression is like linear regression except that we assume our dependent variable $y_i$ is *jointly* predicted by multiple independent variables: 

$x_1$, $x_2$, ..., $x_n$.

Simple linear regression assumes that our data $(x_i, y_i)$ has the following form:

$y_i = \beta_0 + \beta_1 x_i + \epsilon_i$

With multiple regression, we now extend this model to include multiple predictors for our data ($x_{1i}$, $x_{2i}$, ..., $x_{ni}$, $y_i$):

$y_i = \beta_0 + \beta_1 x_{1i} + \beta_2 x_{2i} + \ ... \ + \beta_n x_{ni} + \epsilon_i $

In most cases, multiple regression once again uses the same *Ordinary Least Squares* (OLS) parameter estimation as simple regression. However, interpreting the parameter estimates is a little less straightforward.

*How would we interpret $\beta_0 = 1$, $\beta_1 = 2$, $\beta_2 = 3$?*

(Think of a simple example. What variables would you want to use to predict somebody's height *as accurately as possible*? Now imagine each of those variables is one of your $x_{ji}$ variables.)

## Multiple regression in python

The scikit-learn `LinearRegression` class is very easy to extend to multiple regression!

We'll also demonstrate the `statsmodels` approach below for more robust statistical analysis.

In [ ]:
# scikit learn approach
from sklearn.linear_model import LinearRegression

x_vals = np.array(gap_subset[['logGdpPercap', 'logPop']]) # Note: this double bracket syntax is important!
x_vals = x_vals.reshape(len(gap_subset), 2)
x_vals

y_vals = np.array(gap_subset['lifeExp'])
y_vals

mod = LinearRegression().fit(X = x_vals, y = y_vals)

mod.intercept_
mod.coef_

What do the coefficient estimates above suggest? 

How should we interpret them? 

In [ ]:
# R^2 for our regression
mod.score(X = x_vals, y = y_vals)

In [ ]:
# What if we don't include the population predictor?
x_single = np.array(gap_subset['logGdpPercap']) # Note: this double bracket syntax is important!
x_single = x_single.reshape(len(gap_subset), 1)
x_single

mod_simple = LinearRegression().fit(X = x_single, y = y_vals)
mod_simple.score(X = x_single, y = y_vals)

# Doesn't seem like population helps much...

The statsmodels approach gives us a bit more insight here.

In [ ]:
# statsmodels approach
import statsmodels.formula.api as smf

multiple_reg = smf.ols('lifeExp ~ logGdpPercap + logPop', data = gap_subset).fit()

# View the results
multiple_reg.summary()

# Our population variable does have a significant positive slope, 
# but it's pretty small and the effect may be driven by outliers.

# Regression: wrap-up

Regression is one of the most powerful tools in our data science / analysis toolkit. 

Many problems with understanding and predicting data can be distilled down to basic regression.

However, it's important to be familiar with the *limitations* in regression as well. 

One class of these limitations is when **the data violate the assumptions of linear regression**. We're *not* going to get into these issues in this class, but it's good to be aware that not all data (even data that looks linear!) can be accurately described by linear regression. There are a number of tricks for diagnosing this, such as plotting your residuals (we may explore this on the problem set as a way to cover some of this material).

However, an important limitation that we *will* discuss now is when **regression overfits the data**. Avoiding overfitting is something that arises in virtually all modeling contexts. It's easiest to illustrate with regression, but the things we discuss here will be relevant throughout the remainder of the quarter.

***